In [58]:
class Neo4jConnection:
    def __init__(self, uri, user, pwd):  # Changed _init_ to __init__
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None  # Initialize __driver to None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))  # Use self.__uri, self.__user, self.__pwd and assign to self.__driver
        except Exception as e:
            print("Failed to create the driver:", e)

    def close(self):
        if self.__driver is not None:
            self.__driver.close()

    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try:
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() # Use self.__driver
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally:
            if session is not None:
                session.close()
        return response

    def get_driver(self):
        return self.__driver #Return self.__driver

In [59]:
def populate_neo4j(conn, data):
    # Access the driver using the getter method
    with conn.get_driver().session() as session:  # Use a transaction
        for item in tqdm(data, total=len(data)):  # Iterate through the list of dictionaries
            wall_id = item["wall_id"]
            materials = item["materials"]

            # Create Wall node
            session.run("""
                MERGE (w:Wall {id: $wall_id})
            """, parameters={'wall_id': wall_id})

            # Create Material nodes and relationships
            for material in materials:
                session.run("""
                    MERGE (m:Material {name: $name})
                    ON CREATE SET m.density = $density, m.conductivity = $conductivity,
                                 m.u_value = $u_value, m.embodied_carbon_coefficient = $carbon_coeff,
                                 m.cost = $cost, m.recyclability = $recyclability,
                                 m.bio_based = $bio_based, m.color = $color
                    ON MATCH SET m.density = $density, m.conductivity = $conductivity,
                                 m.u_value = $u_value, m.embodied_carbon_coefficient = $carbon_coeff,
                                 m.cost = $cost, m.recyclability = $recyclability,
                                 m.bio_based = $bio_based, m.color = $color
                    MERGE (w:Wall {id: $wall_id})
                    MERGE (m)-[:USED_IN {thickness: $thickness}]->(w)
                """, parameters={
                    'name': material['material'],
                    'wall_id': wall_id,
                    'thickness': material['thickness'],
                    'density': material['density'],
                    'conductivity': material['conductivity'],
                    'u_value': material['u_value'],
                    'carbon_coeff': material['embodied_carbon_coefficient'],
                    'cost': material['cost'],
                    'recyclability': material['recyclability'],
                    'bio_based': material['bio_based'],
                    'color': material['color'],
                })

            # Create Metric nodes for additional properties and relationships
            metrics = {
                'construction_demolition_waste': item.get('construction_demolition_waste'),
                'circular_economy': item.get('circular_economy'),
                'heritage_preservation': item.get('heritage_preservation'),
                'responsible_material_sourcing': item.get('responsible_material_sourcing'),
                'embodied_ghg_emissions': item.get('embodied_ghg_emissions'),
                'affordable_adoption_high_quality_housing_conditions': item.get('affordable_adoption_high_quality_housing_conditions')
            }

            # Loop over each metric and create nodes and relationships
            for metric_name, metric_value in metrics.items():
                if metric_value is not None:
                    session.run("""
                        MERGE (m:Metric {name: $metric_name})
                        ON CREATE SET m.value = $value
                        ON MATCH SET m.value = $value
                        MERGE (w:Wall {id: $wall_id})
                        MERGE (w)-[:HAS_METRIC {value: $value}]->(m)
                    """, parameters={
                        'metric_name': metric_name,
                        'value': metric_value,
                        'wall_id': wall_id
                    })

In [61]:
import json

if __name__ == "__main__":
    # Path to your JSON file
    file_path = r'D:\Research\wall_population.json'  # Use raw string for Windows paths

    # Load JSON data
    print("Loading data...")
    with open(file_path, 'r') as f:
        data = json.load(f)

    # Neo4j connection details
    uri = "bolt://localhost:7687"  # Replace with your Neo4j URI
    user = "neo4j"                 # Replace with your Neo4j username
    password = "123456789"         # Replace with your Neo4j password

    # Establish connection
    print("Connecting to Neo4j...")
    conn = Neo4jConnection(uri, user, password)

    # Populate Neo4j database
    print("Populating the Neo4j database...")
    try:
        populate_neo4j(conn, data)
    finally:
        # Close connection
        conn.close()
        print("Neo4j database population complete.")


Loading data...
Connecting to Neo4j...
Populating the Neo4j database...


100%|██████████| 2000/2000 [00:50<00:00, 39.56it/s]

Neo4j database population complete.


In [67]:
graph.refresh_schema()
print(graph.schema)

Node properties:
Wall {id: INTEGER}
Material {name: STRING, density: FLOAT, conductivity: FLOAT, u_value: FLOAT, embodied_carbon_coefficient: FLOAT, cost: FLOAT, recyclability: INTEGER, bio_based: BOOLEAN, color: STRING}
Metric {name: STRING, value: FLOAT}
Relationship properties:
USED_IN {thickness: FLOAT}
HAS_METRIC {value: FLOAT}
The relationships:
(:Wall)-[:HAS_METRIC]->(:Metric)
(:Material)-[:USED_IN]->(:Wall)


In [ ]:
cypher = """
  MATCH (n) 
  RETURN count(n)
  """
result = graph.query(cypher)
result

[{'count(n)': 2023}]

In [71]:
cypher = """
  MATCH  (m:Material) 
  RETURN count(m) AS numberOfMaterial 
  """
graph.query(cypher)


[{'numberOfMaterial': 18}]

In [72]:
cypher = """
  MATCH (rockwool:Material {name:"rockwool"}) 
  RETURN rockwool
  """
graph.query(cypher)

[]

In [76]:
cypher_query = """
MATCH (w:Wall {id: $wall_id})-[:HAS_METRIC]->(m:Metric)
RETURN m.name AS metric_name, m.value AS metric_value
"""

parameters = {'wall_id': 1010}  # Replace with the actual wall ID
results = graph.query(cypher_query, parameters)

print(results)

[{'metric_name': 'construction_demolition_waste', 'metric_value': 68.46}, {'metric_name': 'circular_economy', 'metric_value': 91.29}, {'metric_name': 'heritage_preservation', 'metric_value': 0.0}, {'metric_name': 'responsible_material_sourcing', 'metric_value': 100.0}, {'metric_name': 'embodied_ghg_emissions', 'metric_value': 93.69}]


In [17]:
from datasets import load_dataset, Dataset

# Load your custom JSON dataset
data = [
    {"natural_query": "Find all nodes connected to node A.", "cypher_query": "MATCH (a {name: 'A'})--(b) RETURN b;"},
    {"natural_query": "Get the shortest path between node A and node B.", "cypher_query": "MATCH p=shortestPath((a {name: 'A'})-[*]-(b {name: 'B'})) RETURN p;"}
]

dataset = Dataset.from_list(data)

# Split into train and validation sets
dataset = dataset.train_test_split(test_size=0.2)
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['natural_query', 'cypher_query'],
        num_rows: 1
    })
    test: Dataset({
        features: ['natural_query', 'cypher_query'],
        num_rows: 1
    })
})


In [ ]:
q_one = "What are the materials in wall 1010?"
q_two = "What is the cricluar economic score for wall 1010?"

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.llms import Ollama
from langchain_core.tools import Tool # Make sure to import Tool 
# Import the create_react_agent and AgentExecutor functions 
from langchain.agents import AgentExecutor, create_react_agent,AgentOutputParser  # Make sure you import the hub object for pulling prompts 
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.schema import StrOutputParser
from langchain_community.chat_message_histories import Neo4jChatMessageHistory
from langchain_community.graphs import Neo4jGraph
from uuid import uuid4
import streamlit as st
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = 'True'
os.environ["LANGCHAIN_API_KEY"] = str(os.getenv("LANGCHAIN_API_KEY"))

# Neo4j Connection Class
class Neo4jConnection:
    def __init__(self, uri, user, password):
        self._driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        if self._driver:
            self._driver.close()

    def query(self, query, parameters=None):
        with self._driver.session() as session:
            return session.run(query, parameters).data()
# Define the prompt template for conversation
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a Neo4j expert having a conversation about Cypher queries"),
        ("human", "{input}"),
    ]
)

# Set up Neo4j Graph connection
graph = Neo4jGraph(
    url="bolt://localhost:7687",
    username="neo4j",
    password="123456789"
)

# Questions
q_one = "What are the materials in wall 1010?"
q_two = "What is the circular economic score for wall 1010?"

# Ollama LLaMA2 LLM
llm = Ollama(model="llama2")

# Create Cypher chat function using StrOutputParser
output_parser = StrOutputParser()
cypher_chat = prompt | llm | output_parser

# Streamlit input handling (if used with Streamlit)
if input_text:  # Assuming `input_text` is a user input from Streamlit
    st.write(cypher_chat.invoke({"input": input_text}))  # Ensure "input" matches the template

# Response for q_one
response_one = cypher_chat.invoke({"input": q_one})
response_two = cypher_chat.invoke({"input": q_two})

# Print results
print("Response for Q1:", response_one)

print("\nResponse for Q2:", response_two)





Response for Q1: 
Expert: Ah, an excellent question! *adjusts glasses* Wall 1010, you say? Let me just check our lovely graph... *consults query results*

Well, it looks like Wall 1010 is made up of a variety of materials. *scrolls through results* We have wood, concrete, steel, and even some interesting blends of materials. *glances around* It's quite the mix!

Would you like me to drill down further into any of these materials? Perhaps you're interested in knowing more about the types of wood used? Or maybe you want to see which companies produced the concrete and steel? Just let me know, my friend! *smiles*

Response for Q2: Expert: Ah, a query about Cypher! *adjusts glasses* The circular economic score for wall 1010? Let me see... *checks graph database* Hmm, I'm afraid that information isn't stored in the graph directly. However, I can help you find out by querying the relevant data sources.

May I ask what you need to know this score for? Are you working on a project related to s

ImportError: cannot import name 'BaseMode' from 'pydantic' (c:\Users\Lenovo\.conda\envs\First_env\Lib\site-packages\pydantic\__init__.py)

In [5]:
import os
from typing import List
from langchain.chains.openai_functions import create_structured_output_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.llms import Ollama

# Enable LangChain tracing and set API key
os.environ["LANGCHAIN_TRACING_V2"] = 'True'
os.environ["LANGCHAIN_API_KEY"] = str(os.getenv("LANGCHAIN_API_KEY"))

# Initialize the LLM
llm = Ollama(model="llama2")

# Define entities specific to this dataset
class WallEntities(BaseModel):
    """Identifying information about walls and materials."""
    materials: List[str] = Field(
        ..., description="List of material names mentioned in the text."
    )
    wall_ids: List[int] = Field(
        ..., description="List of wall IDs mentioned in the text."
    )
    properties: List[str] = Field(
        ..., description="List of properties (e.g., recyclability, embodied carbon) mentioned."
    )

class WallDataPipeline:
    def __init__(self, llm, data):
        self.llm = llm
        self.data = data  # JSON data loaded into memory

    # Step 1: Entity extraction
    def prepare_entity_chain(self):
        entity_chain_prompt = ChatPromptTemplate.from_messages(
            [
                ("system", "You extract materials, wall IDs, and properties from user queries."),
                ("human", "Extract entities from this query: {query}"),
            ]
        )
        entity_chain = create_structured_output_chain(WallEntities, self.llm, entity_chain_prompt)
        return entity_chain

    # Step 2: Map entities to the dataset
    def map_to_data(self, entities: WallEntities):
        """Map extracted entities to the JSON dataset."""
        results = []

        # Find walls by ID
        for wall_id in entities.wall_ids:
            wall = next((w for w in self.data if w["wall_id"] == wall_id), None)
            if wall:
                results.append({"wall_id": wall_id, "data": wall})

        # Find materials
        for material in entities.materials:
            for wall in self.data:
                for mat in wall["materials"]:
                    if material.lower() in mat["material"].lower():
                        results.append({"wall_id": wall["wall_id"], "material": mat})

        # Find walls by properties
        for property_name in entities.properties:
            for wall in self.data:
                if property_name in wall:
                    results.append({"wall_id": wall["wall_id"], "property": {property_name: wall[property_name]}})

        return results

    # Step 3: Generate responses
    def prepare_response_prompt(self):
        response_template = """Based on the question, generate a response using the matched data:
        Question: {query}
        Matched Data: {data}"""
        response_prompt = ChatPromptTemplate.from_messages(
            [
                ("system", "Generate a natural language response."),
                ("human", response_template),
            ]
        )
        return response_prompt

    def prepare_pipeline(self):
        entity_chain = self.prepare_entity_chain()
        response_prompt = self.prepare_response_prompt()

        pipeline = (
            RunnablePassthrough.assign(entities=entity_chain)
            | RunnablePassthrough.assign(matched_data=lambda x: self.map_to_data(x["entities"]))
            | response_prompt
            | self.llm
            | StrOutputParser()
        )

        return pipeline

    def run_pipeline(self, query):
        entity_chain = self.prepare_entity_chain()
        entities = entity_chain.run({"query": query})
        matched_data = self.map_to_data(entities)
        response_prompt = self.prepare_response_prompt()
        response = response_prompt.run({"query": query, "data": matched_data})
        return response


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_10176\3304141224.py:15: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama2")
